In [ ]:
#defining file path for test dataset
dataset = "IN-Abs" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "/kaggle/working/output/"

In [ ]:
cd /kaggle/input/utilis

In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import glob
import sys
sys.path.insert(0, '../')
from utilities import *
import os
import nltk

In [ ]:
#creating output directory if it doesnt exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
names, data_source, data_summary = get_summary_data(dataset, "test")

In [ ]:
cd /kaggle/input/

In [ ]:
#references is a list containg all the summaries in the test dataset
references=list()
for reference_summary in data_summary:
    references.append(reference_summary)

In [ ]:
summarylength=list()
docid=list()
for i in references:
    summarylength.append(len(i.split(" ")))
for i in range(1,len(summarylength)+1):
    docid.append(i)

In [ ]:
# print(summarylength)
# print(docid)

In [ ]:
import matplotlib.pyplot as plt

avg_summary_length = np.mean(summarylength)

# Plotting
plt.figure(figsize=(6, 4))  # Adjust figure size if needed
plt.bar(docid, summarylength, color='skyblue', label='Summary Length')
plt.axhline(y=avg_summary_length, color='red', linestyle='--', label=f'Average: {avg_summary_length:.2f}')
plt.xlabel('Document ID')
plt.ylabel('Summary Length')
plt.ylim(0, 8000)
plt.title('Summary Lengths for Different Documents')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.legend()
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
total_length = sum(len(string.split(" ")) for string in references)
l=len(references[0].split(" "))
for i in references:
    l=max(l,len(i.split(" ")))
print(l)
print(references[0])
# Calculate average length
average_length = total_length / len(references)
print(average_length)

In [ ]:
cd /kaggle/input/bart-utilities

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, BartForConditionalGeneration, BartTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig
from BART_utilities import *
#loading gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned pegasus model
pegasus_tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus") 
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/output-model")
pegasus_model.to(device)


# Loading Bart Model and tokenizer



base_bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
base_bart_model.to(device)

bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)
new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']
special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = bart_tokenizer.add_special_tokens(special_tokens_dict)
base_bart_model.resize_token_embeddings(len(bart_tokenizer))
print(len(bart_tokenizer))

# Load the fine-tuned bart model
bart_model = LitModel(learning_rate = 2e-5, tokenizer = bart_tokenizer, model = base_bart_model)
bart_model.load_state_dict(torch.load("/kaggle/input/bartmodel/outputBARTModel (1).pt"))
bart_model.to(device)

In [ ]:
from difflib import SequenceMatcher

def similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

In [ ]:
def ensemble_summarizer(text):
# Generate summary using Legal PEGASUS
    pegasus_input_tokenized = pegasus_tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    pegasus_input_tokenized = pegasus_input_tokenized.to(device)  # Move input tensor to GPU if available
    # Ensure the model and input tensors are on the same device
    with torch.no_grad():
        pegasus_summary_ids = pegasus_model.generate(pegasus_input_tokenized,
                                                num_beams=9,
                                                no_repeat_ngram_size=3,
                                                length_penalty=2.0,
                                                min_length=200,
                                                max_length=1000,
                                                early_stopping=True)
    pegasus_summary = [pegasus_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in pegasus_summary_ids][0]
   

    
    # Generate summary using BART
    
    bart_input_tokenized = bart_tokenizer.encode(text, truncation=True, return_tensors='pt')
    bart_input_tokenized = bart_input_tokenized.to(device)
    bart_summary_ids = bart_model.model.to(device).generate(bart_input_tokenized,
                                  length_penalty=0.01,
                                  min_length=200,
                                  max_length=1000)
    bart_summary = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in bart_summary_ids][0]

  
    
    
    bart_legal_similarity = similarity_score(bart_summary, pegasus_summary)
    
    # Select the summary with higher similarity or a specific threshold
    if bart_legal_similarity >= 0.7:  # Adjust the threshold as needed
        ensemble_summary = bart_summary  # Select BERT summary
    else:
        # If the similarity is low, choose the longer summary as the ensemble result
        if len(bart_summary) >= len(pegasus_summary):
            ensemble_summary = bart_summary
        else:
            ensemble_summary =pegasus_summary
    
    return ensemble_summary
    
    
#     # Voting mechanism: Choose the summary with the highest score (simple voting)
#     # Here, we are using a simple approach where we count the number of words in the summaries
#     pegasus_words = len(pegasus_summary.split())
#     bart_words = len(bart_summary.split())

#     # Choose the summary with the higher word count
#     final_summary = pegasus_summary if pegasus_words > bart_words else bart_summary

#     return final_summary


Evaluating the model

In [ ]:
!pip install evaluate
!pip install rouge_score

In [ ]:
from evaluate import load
# Load the ROUGE metric
import evaluate

candidates=list()


for text in data_source[:5]:
    candidates.append(ensemble_summarizer(text))
    
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=candidates, references=references[:5])
print(results)
    

In [ ]:
from nltk.translate.bleu_score import corpus_bleu



# Calculate BLEU scores
bleu = corpus_bleu(references[:5], candidates)

print(f'Ensemble BLEU Score: {bleu:.4f}')
